In [1]:
using CSV
using DataFrames

In [65]:
stock_data = CSV.read("top_stocks_data.csv", DataFrame)
stocks = unique(stock_data, :ticker)[!,:ticker]
μ = []
for stock in stocks
    expected_return = 0
    timepoints = subset(stock_data, :ticker => ByRow(==(stock)))
    for time in eachrow(timepoints)
        ret = (time[:open] - time[:close])/time[:open]
        expected_return += ret
    end
    expected_return /= 100
    push!(μ, expected_return)
end

In [ ]:
using Statistics
stock_data = sort(stock_data, [:ticker, :timestamp])

function compute_returns(df)
    df = sort(df, :timestamp)
    returns = [missing; diff(df.close) ./ df.close[1:end-1]]
    df[!, :return] = returns
    return df
end

returns_data = combine(groupby(stock_data, :ticker), compute_returns)

returns_wide = unstack(returns_data, :timestamp, :ticker, :return)

returns_wide_clean = dropmissing(returns_wide)

returns_matrix = Matrix(returns_wide_clean[:, Not(:timestamp)])  # Only the returns, no timestamp
Σ = cov(returns_matrix)

7774×8 DataFrame
  Row │ ticker   timestamp            open       high       low        close      volume    return            
      │ String7  String31             Float64    Float64    Float64    Float64    Int64     Float64?          
──────┼───────────────────────────────────────────────────────────────────────────────────────────────────────
    1 │ AAPL     2025-04-21 18:00:00   192.97     230.19     192.3      192.526      14864  missing           
    2 │ AAPL     2025-04-21 18:30:00   192.52     193.16     192.52     192.82       10958        0.00152863
    3 │ AAPL     2025-04-21 19:00:00   192.75     192.89     192.56     192.7         9030       -0.000622342
    4 │ AAPL     2025-04-21 19:30:00   192.69     194.0      192.61     194.0        49735        0.00674624
    5 │ AAPL     2025-04-22 04:00:00   193.79     195.44     193.79     195.09       25649        0.00561856
    6 │ AAPL     2025-04-22 04:30:00   195.09     195.66     195.06     195.33       15756        0.00

In [68]:
using JuMP, Ipopt

λ = 0.5  # risk-return balance parameter

model = Model(Ipopt.Optimizer)

@variable(model, x[1:78] >= 0)
@constraint(model, sum(x) == 1)

@objective(model, Min, -μ'*x + λ*(x'*Σ*x) )

optimize!(model)

println("Optimal allocation: ", value.(x))
println("Max_val ", maximum(value.(x)))
println("Optimal Objective: ", objective_value(model))

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       78
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     3081

Total number of variables............................:       78
                     variables with only lower bounds:       78
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.7656525e-04 2.20e-01 3.96e-03  -1.0 0.00e+00    -  0.00e+00 0.00e+00 